# Build LLM applications with **Haystack**

Haystack Concepts we will cover:

- Nodes
- Pipelines
- (Agents)
- (Document-Store)

In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

## Creating our Knowledge-Base : Indexing Pipeline

In [2]:
!haystack --version

haystack, version 1.21.2


In [ ]:
from haystack.pipelines import Pipeline
from haystack.nodes import Crawler, EmbeddingRetriever
from haystack.document_stores import ElasticsearchDocumentStore
from helper_functions.preprocessor import CustomPreProcessor

# Init documentstore with custom 
mapping = {
    "mappings": {
        "properties": {
            "embedding": {"type": "dense_vector", "dims": 384},
            "authors": {"type": "keyword"},
            "title": {"type": "keyword"},
            "date": {
                "type":   "date",
                "format": "dd.MM.yyyy"
            }
        }
    }
}

document_store = ElasticsearchDocumentStore(index="blogs_clean1", custom_mapping=mapping)

# Define nodes
crawler = Crawler(
    urls=["https://www.inovex.de/de/blog/perspective-dialogue-summarization-with-neural-networks/"],   # Websites to crawl
    filter_urls=["https://www.inovex.de/de/blog/"],
    crawler_depth=1,    # How many links to follow
    output_dir="data/blogs_clean1",  # The directory to store the crawled files, not very important, we don't use the files in this example
)

preprocessor = CustomPreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=50,
)

retriever = EmbeddingRetriever(
        embedding_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        document_store=document_store,
)

# Define pipeline
indexing_pipeline = Pipeline()
indexing_pipeline.add_node(component=crawler, name="crawler", inputs=['File'])
indexing_pipeline.add_node(component=preprocessor, name="preprocessor", inputs=['crawler'])
indexing_pipeline.add_node(component=retriever, name="EmbeddingRetriever", inputs=["preprocessor"])
indexing_pipeline.add_node(component=document_store, name="document_store", inputs=['EmbeddingRetriever'])

In [4]:
docs = indexing_pipeline.run()

Clean https://www.inovex.de/de/blog/perspective-dialogue-summarization-with-neural-networks/
Ignore https://www.inovex.de/de/blog/
Clean https://www.inovex.de/de/blog/industrielle-bildverarbeitung-grundlagen-anwendungen-und-vorteile/
Ignore https://www.inovex.de/de/blog/author/tnguyen/
Clean https://www.inovex.de/de/blog/voltage-and-variables-exploring-forecasting-for-electricity-consumption-through-time-series-analysis-with-the-prophet-model/
Clean https://www.inovex.de/de/blog/self-service-ai-for-everyone-a-comparison-of-automl-services/


Preprocessing: 100%|██████████| 4/4 [00:00<00:00, 40.19docs/s]


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
# Print all documents that have been created
pp.pprint(docs)

{   'documents': [   <Document: {'content': 'This article will elaborate a method for generating abstractive perspective dialogue summarization. Unlike regular dialogue summarization, perspective summarizations aim to outline the point of view of each participant within a dialogue. This work provides an approach to fit datasets intended for regular dialogue summarization to the task of perspective summarizations. It furthermore presents an architecture that can be a solid foundation for this task.\n\nIntroductionDefining summarizationMonologue summarizationDialogue summarizationPerspective dialogue summarizationEstablished dialogue summarization methodsData pre-processingDialogSum datasetAcquiring perspective summary annotationsCleaning and correcting the labelsAssigning the labels to the corresponding speakerArchitectureMulti-head encoderTrainingLoss functionSetupResultsDiscussion and future workChallengesFuture workConclusion\nIntroduction\nFor centuries humans have been living in a 

In [131]:
print("Document (Snippet) Count:", len(document_store.get_all_documents()))

Document (Snippet) Count: 185


## Question Answering : Query PIpeline

In [ ]:
from haystack.nodes import EmbeddingRetriever, FARMReader, BM25Retriever
from haystack import Pipeline
from haystack.utils import print_answers
from haystack.document_stores import ElasticsearchDocumentStore

mapping = {
    "mappings": {
        "properties": {
            "embedding": {"type": "dense_vector", "dims": 384},
            "authors": {"type": "keyword"},
            "title": {"type": "keyword"},
            "date": {
                "type":   "date",
                "format": "dd.MM.yyyy"
            }
        }
    }
}

# Connect documentstore
document_store = ElasticsearchDocumentStore(host="172.17.0.1", index="blogs_clean1", port=9200, custom_mapping=mapping)

# Define nodes
retriever = EmbeddingRetriever(
        embedding_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        document_store=document_store,
)

# Define pipeline
query_pipe = Pipeline()
query_pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])    # Searches for relevant `documents`

In [11]:
retrieved_docs = query_pipe.run(
    query="Tell me about Prophet’s Hyperparameter Tuning?", params={"Retriever": {"top_k": 3}}
)

# print_answers(prediction, details="all")
for idx, doc in enumerate(retrieved_docs["documents"]):
    print(f"{idx}. " + doc.content + "\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0. Overall, Prophet’s ease of interpretation and the relatively small number of adjustments to its default hyperparameters contribute positively to its appeal and make it a strong choice for forecast applications in the electricity consumption context.

1. Iterating across all cut-off points and calculating the average of all computed error metrics, a comprehensive understanding of the forecast performance over time is attained.
Problems faced with Prophet’s Hyperparameter Tuning
In our application case, we need to deal with numerous time series xm,m∈Mx_m, m \in M. Our goal is to identify a parameter setting ˆω∈Ω\hat\omega \in \Omega for a single Prophet model that fits all these time series as best as possible, e.g. this model with parameter settings ˆω\hat\omega ensures a reasonable forecast performance for all these time series recorded by the measurement points MM. We assume that the time series of the measurement points MM represent the unseen target data in their characteristics.

### We can do better - Integrating GPT

In [12]:
import os
from dotenv import load_dotenv
from haystack.nodes import PromptModel, PromptNode

load_dotenv("./.env")

api_key = os.environ.get("AZURE_API_KEY")
deployment_name = os.environ.get("AZURE_DEPLOYMENT_NAME")
base_url = os.environ.get("AZURE_BASE_URL")

# Init Model - Connects to Azure
azure_model = PromptModel(
    model_name_or_path="gpt-35-turbo",
    api_key=api_key,
    model_kwargs={
        "azure_deployment_name": deployment_name,
        "azure_base_url": base_url,
    },
)

# Init PromptNode
prompt_node = PromptNode(model_name_or_path=azure_model)

In [13]:
# Example: Test PromptNode

# Construct Message
messages = [{"role": "system", "content": "You are a helpful assistant"}]
messages.append({"role": "user", "content": "Tell me 1 sentence about haystack by deepset?"})

# Call PromptNode -> Calls OpenAI/Azure API
result = prompt_node(messages)
result[0]

'Haystack by deepset is an open-source framework that simplifies the development of scalable and efficient question-answering systems.'

In [ ]:
from haystack.pipelines import Pipeline
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

document_store = ElasticsearchDocumentStore(index="blogs_clean1", custom_mapping=mapping)

# Create nodes
retriever = EmbeddingRetriever(
    embedding_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    document_store=document_store,
)


# PromptTemplate adds additional context to PromptNode
qa_prompt = PromptTemplate(
    prompt="""Given the context, provide a short consise answer to the question.
                Context: {join(documents)}; 
                Question: {query}; 
                Answer:""",
    output_parser=AnswerParser(),
)

# Combine Azure Model with Prompt
prompt_node = PromptNode(
    model_name_or_path=azure_model,
    default_prompt_template=qa_prompt
)


# Create Pipeline
inovex_query_pipe = Pipeline()
inovex_query_pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
inovex_query_pipe.add_node(component=prompt_node, name="PromptNode", inputs=["Retriever"])

In [15]:
# Execute pipeline

output = inovex_query_pipe.run(query="Tell me about Prophet’s Hyperparameter Tuning?", params={"Retriever": {"top_k": 3}})
# print(output["answers"][0].answer)
print_answers(output, details="minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Query: Tell me about Prophet’s Hyperparameter Tuning?'
'Answers:'
[   {   'answer': 'Prophet offers cross-validation functionalities to quickly '
                  'and easily find the best working forecast model across '
                  'various hyperparameters. It utilizes the simulated '
                  'historical forecast, which is based on a rolling origin '
                  'evaluation procedure for assessing the quantitative '
                  'performance of time series forecasts. The goal is to '
                  'identify a parameter setting for a single Prophet model '
                  'that fits all time series recorded by the measurement '
                  'points as best as possible.'}]


### with Reader model: (We could skip the Reader model altogether, as its functionality is replaced by gpt)

In [39]:
from haystack.nodes import EmbeddingRetriever, FARMReader, BM25Retriever
from haystack import Pipeline
from haystack.utils import print_answers
from haystack.document_stores import ElasticsearchDocumentStore

mapping = {
    "mappings": {
        "properties": {
            "embedding": {"type": "dense_vector", "dims": 384},
            "authors": {"type": "keyword"},
            "title": {"type": "keyword"},
            "date": {
                "type":   "date",
                "format": "dd.MM.yyyy"
            }
        }
    }
}

# Connect documentstore
document_store = ElasticsearchDocumentStore(index="blogs_clean1", custom_mapping=mapping)

# Define nodes
retriever = EmbeddingRetriever(
        embedding_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        document_store=document_store,
)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

# Define pipeline
query_pipe = Pipeline()
query_pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])    # Searches for relevant `documents`
query_pipe.add_node(component=reader, name="Reader", inputs=["Retriever"])      # Extract top answers from retrieved documents

Cannot validate index for custom mappings. Skipping index validation.


In [40]:
prediction = query_pipe.run(
    query="Tell me about Prophet’s Hyperparameter Tuning?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

print_answers(prediction, details="all")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:06<00:00,  6.26s/ Batches]

'Query: Tell me about Prophet’s Hyperparameter Tuning?'
'Answers:'
[   <Answer {'answer': 'Problems faced with Prophet’s Hyperparameter TuningAlternative evaluation strategy using Prophet for time series analysisAnalysis of error statistics and plotsExperiences using Prophet as a forecasting model for electricity consumption\nWhat is Prophet?\nConstructing time series forecasting requires a sophisticated set of methods to capture the complex patterns of time-dependent data. In 2017, researchers from Facebook presented their open-source project named Prophet, a framework tailored for modeling time series in Python or R. in their paper “Forecasting at Scale“. It stands out for its user-friendly simplicity', 'type': 'extractive', 'score': 0.3024429380893707, 'context': 'gProblems faced with Prophet’s Hyperparameter TuningAlternative evaluation strategy using Prophet for time series analysisAnalysis of error statistics and plotsExperiences using Prophet as a forecasting model for electrici

In [43]:
print_answers(prediction, details="minimum")

'Query: Tell me about Prophet’s Hyperparameter Tuning?'
'Answers:'
[   {   'answer': 'Problems faced with Prophet’s Hyperparameter '
                  'TuningAlternative evaluation strategy using Prophet for '
                  'time series analysisAnalysis of error statistics and '
                  'plotsExperiences using Prophet as a forecasting model for '
                  'electricity consumption\n'
                  'What is Prophet?\n'
                  'Constructing time series forecasting requires a '
                  'sophisticated set of methods to capture the complex '
                  'patterns of time-dependent data. In 2017, researchers from '
                  'Facebook presented their open-source project named Prophet, '
                  'a framework tailored for modeling time series in Python or '
                  'R. in their paper “Forecasting at Scale“. It stands out for '
                  'its user-friendly simplicity',
        'context': 'gProblems faced wi

## Multi-turn Conversations : Introducting Agents

### Setup Agent's Tools

- inovex_query_pipeline (from before)
- game_query_pipeline (Pipeline accesses 'Game of Thrones' database)

In [16]:
# Add PromptTemplate & Define Pipeline
from haystack.pipelines import Pipeline
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

# Init database
document_store = ElasticsearchDocumentStore(index="blogs_clean1", custom_mapping=mapping)

# Create nodes
# Create PromptTemplate with additinal context send PromptModel
qa_prompt = PromptTemplate(
    prompt="""Given the context, answer the question in 1 or 2 sentences.
                Context: {join(documents)}; 
                Question: {query}; 
                Answer:""",
    output_parser=AnswerParser(),
)

# Combine PromptModel & PromptTemplate
prompt_node = PromptNode(
    model_name_or_path=azure_model,
    default_prompt_template=qa_prompt
)

retriever = EmbeddingRetriever(
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    document_store=document_store,
)

# Create Pipeline
game_prompt_pipeline = Pipeline()
game_prompt_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
game_prompt_pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Retriever"])

### Initialize Agent

In [17]:
from haystack.agents.base import Tool
from haystack.agents.conversational import ConversationalAgent
from haystack.agents.memory import ConversationSummaryMemory

inovex_blog_crawler_tool = Tool(
    name="inovex_blog_crawler",
    pipeline_or_node=inovex_query_pipe,
    description="useful for when you need to find content from the inovex blog", # agent uses this for its decision!
    output_variable="answers",
)

got_qa_tool = Tool(
    name="games_of_thrones_QA",
    pipeline_or_node=game_prompt_pipeline,
    description="useful for when you need to answer questions about games of thrones",
    output_variable="answers",
)

tools = [inovex_blog_crawler_tool, got_qa_tool]

In [ ]:

conversational_agent_prompt_node = PromptNode(
    model_name_or_path=azure_model,
    max_length=256,
    top_k=2,
    stop_words=["Observation:"], # react framework
    model_kwargs={"temperature": 0.5, "top_p": 0.9}
)

memory = ConversationSummaryMemory(conversational_agent_prompt_node, summary_frequency=2)

zero_shot_agent_template = PromptTemplate("deepset/zero-shot-react")

agent = ConversationalAgent(
    prompt_node=conversational_agent_prompt_node, prompt_template=zero_shot_agent_template, tools=tools, memory=memory
)


In [ ]:
res_crawl = agent.run("What can you tell me about prophets hyptertuning ?")

In [ ]:
pp.pprint(res_crawl)

In [21]:
res_crawl['answers'][0].answer

"Prophet's hyperparameter tuning involves utilizing cross-validation functionalities to find the best working forecast model across variousFinal Answer: Prophet's hyperparameter tuning involves utilizing cross-validation functionalities to find the best working forecast model across various hyperparameters. hyperparameters."

In [ ]:
res_got = agent.run("Who is the Son of Eddard?")

In [25]:
res_got['answers'][0].answer

'Robb Stark.Final Answer: Robb Stark.'